In [5]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

import random
import pickle

import datalab.storage as gcs
import pandas as pd

In [1]:
spark.read.load('gs://project_joyce_scribo/all_texts').

DataFrame[source: string, text: string]

In [12]:
items = gcs.Bucket('project_joyce').items()

texts = []
for item in items:
    if item.key.endswith('.txt'):
        filename = item.key.split('/')[-1]
        %gcs read --object 'gs://project_joyce/'$item.key --variable text
        print('Read', filename)
        text = text.decode('utf-8')
        # common replacements, idk how to deal with py2 encoding...
        texts.append((item.key, text))

Read A_Book_of_Common_Prayer_Didion_Joan.txt
Read A_Little_Life_A_Novel_by_Hanya_Yanagihara.txt
Read A_Mercy_Toni_Morrison.txt
Read A_Supposedly_Fun_Thing_I_ll_Never_Do_Aga_Wallace_David_Foster.txt
Read Amsterdam_Ian_McEwan_1998_.txt
Read Atonement_Ian_McEwan_2001_.txt
Read Beckett_Samuel_Beckett_Remembering_Remembering_Beckett_Arcade_2011_.txt
Read Beckett_Samuel_Collected_Poems_in_English_and_French_Grove_1977_.txt
Read Beckett_Samuel_Collected_Shorter_Plays_Grove_1984_.txt
Read Beckett_Samuel_Company_Ill_Seen_Ill_Said_etc._Faber_Faber_2009_.txt
Read Beckett_Samuel_Complete_Dramatic_Works_Faber_1986_.txt
Read Beckett_Samuel_Complete_Short_Prose_1929_1989_Grove_1995_.txt
Read Beckett_Samuel_Disjecta_Grove_1984_.txt
Read Beckett_Samuel_Dream_of_Fair_to_Middling_Women_Arcade_1993_.txt
Read Beckett_Samuel_Echo_s_Bones_Grove_2014_.txt
Read Beckett_Samuel_Endgame_Act_Without_Words_I_Grove_1958_.txt
Read Beckett_Samuel_Ends_and_Odds_Grove_1981_.txt
Read Beckett_Samuel_First_Love_Other_Short

Read collected_stories_gabriel_garcia_marquez.txt
Read complete_works_ernest_hemingway.txt
Read complete_works_f_scott_fitzgerald.txt
Read complete_works_virginia_woolf.txt
Read in_evil_hour_gabriel_garcia_marquez.txt
Read leaf_storm_gabriel_garcia_marquez.txt
Read living_to_tell_the_tale_gabriel_garcia_marquez.txt
Read love_in_the_time_of_cholera_gabriel_garcia_marquez.txt
Read memories_of_my_melancholy_whores_gabriel_garcia_marquez.txt
Read news_of_a_kidnapping_gabriel_garcia_marquez.txt
Read no_one_writes_to_the_colonel_gabriel_garcia_marquez.txt
Read of_love_and_other_demons_gabriel_garcia_marquez.txt
Read one_hundred_hears_of_solitude_gabriel_garcia_marquez.txt
Read strange_pilgrims_gabriel_garcia_marquez.txt
Read the_autumn_of_the_patriarch_gabriel_garcia_marquez.txt
Read the_general_in_his_labirinth_gabriel_garcia_marquez.txt
Read the_last_interview_and_other_conversations_gabriel_garcia_marquez.txt
Read the_story_of_shipwrecked_sailor_gabriel_garcia_marquez.txt
Read wolves_vs_h

In [13]:
sc.parallelize(texts).toDF(['source', 'text']) \
    .write.mode('overwrite').parquet('gs://project_joyce/all_texts')

In [44]:
unicode_dict = [
  (u'\u2019', '\''),
  (u'\u2014', '--'),
  (u'\u201d', '"'),
  (u'\u201c', '"'),
  (u'\u2026', '...'),
  (u'\u2018', '\''),
  (u'\u2013', '-'),
  (u'\x91', '\''),
  (u'\x92', '\''),
  (u'\u2011', '-'),
  ('\'\'', '"'),
  ('``', '"'),
]
  
def fix_unicode(text):
  for uni, replacement in unicode_dict:
    text = text.replace(uni, replacement)
  return text

def tokenize(text):
    import nltk
    nltk.download('punkt', '.')
    tokens = []
    for token in nltk.word_tokenize(text):
      if token == '\'\'':
        tokens.append('"')
      elif token == '``':
        tokens.append('"')
      else:
        tokens.append(token)
    return tokens

def split_long(text, num_white_words=100000):
    whitespace_split = fix_unicode(text).split()
    texts = []
    for i in range(int(len(whitespace_split) / num_white_words) + 1):
        texts.append(' '.join(whitespace_split[i * num_white_words: min((i + 1) * num_white_words, len(whitespace_split))]))
    return texts

def random_segments(tokens, low=20, high=100):
    index = 0
    phrases = []
    while(True):
        length = random.randint(low, high)
        phrases.append(tokens[index : index + length])
        index += int(length / 2)
        if length + index >= len(tokens):
            break
    return phrases

In [45]:
spark.read.load('gs://project_joyce/all_texts').repartition(200) \
    .withColumn('long_split', F.explode(F.udf(split_long, ArrayType(StringType()))(F.col('text')))).drop('text').repartition(200) \
    .withColumn('tokens', F.udf(tokenize, ArrayType(StringType()))(F.col('long_split'))).drop('long_split') \
    .withColumn('phrase', F.explode(F.udf(random_segments, ArrayType(ArrayType(StringType())))(F.col('tokens')))).drop('tokens') \
      .write.mode('overwrite').parquet('gs://project_joyce/phrases')

In [46]:
spark.read.load('gs://project_joyce/phrases').sample(False, 0.0001).take(1000)

[Row(source='A_Little_Life_A_Novel_by_Hanya_Yanagihara.txt', phrase=['cried', 'for', 'himself', '.', 'How', 'was', 'he', 'going', 'to', 'keep', 'living', 'without', 'Willem', '?', 'His', 'entire', 'life', '--', 'his', 'life', 'after', 'Brother', 'Luke', ',', 'his', 'life', 'after', 'Dr.', 'Traylor']),
 Row(source='Nabokov_Vladimir_Invitation_to_a_Beheading.txt', phrase=['it', 'would', 'certainly', 'be', 'that', 'great', 'artist', 'rather', 'than', 'G.', 'H.', 'Orwell', 'or', 'other', 'popular', 'purveyors', 'of', 'illustrated', 'ideas', 'and', 'publicistic', 'fiction', '.', 'Incidentally', ',', 'I', 'could', 'never', 'understand', 'why', 'every', 'book', 'of', 'mine', 'invariably', 'sends', 'reviewers', 'scurrying', 'in', 'search', 'of', 'more', 'or', 'less', 'celebrated', 'names', 'for', 'the', 'purpose', 'of', 'passionate', 'comparison', '.', 'During', 'the', 'last', 'three', 'decades', 'they', 'have', 'hurled', 'at', 'me', '(', 'to', 'list', 'but', 'a', 'few', 'of', 'these', 'harmle

In [ ]:
items = gcs.Bucket('project_joyce_scribo').items()

scribo_texts = []
for item in items:
    if item.key.endswith('.txt'):
        filename = item.key.split('/')[-1]
        %gcs read --object 'gs://project_joyce_scribo/'$item.key --variable text
        print('Read', filename)
        text = text.decode('utf-8')
        # common replacements, idk how to deal with py2 encoding...
        scribo_texts.append((item.key, text))

Read scribophile_texts0.txt
Read scribophile_texts1.txt
Read scribophile_texts2.txt
Read scribophile_texts3.txt
Read scribophile_texts4.txt
Read scribophile_texts5.txt
Read scribophile_texts6.txt
Read scribophile_texts7.txt
Read scribophile_texts8.txt


In [ ]:
sc.parallelize(scribo_texts).toDF(['source', 'text']).write.mode('overwrite').parquet('gs://project_joyce_scribo/all_texts')

In [47]:
spark.read.load('gs://project_joyce_scribo/all_texts').repartition(200) \
    .withColumn('long_split', F.explode(F.udf(split_long, ArrayType(StringType()))(F.col('text')))).drop('text').repartition(200) \
    .withColumn('tokens', F.udf(tokenize, ArrayType(StringType()))(F.col('long_split'))).drop('long_split') \
    .withColumn('phrase', F.explode(F.udf(random_segments, ArrayType(ArrayType(StringType())))(F.col('tokens')))).drop('tokens') \
      .write.mode('overwrite').parquet('gs://project_joyce_scribo/phrases')

In [48]:
spark.read.load('gs://project_joyce_scribo/phrases').sample(False, 0.01).take(10000)

[Row(source='scribophile_texts1.txt', phrase=['those', 'instances', 'of', 'brutal', ',', 'honest', 'vulnerability', 'that', 'made', 'Cody', 'mine', 'more', 'than', 'anything', 'else', '.', 'The', 'side', 'of', 'Cody', 'I', 'now', 'know', 'no', 'one', 'ever', 'saw', ',', 'ever', 'truly', 'got', 'to', 'know', '.', 'The', 'sad', 'side', 'of', 'him', ',', 'the', 'contemplative', 'side', ',', 'the', 'side', 'that', 'worried', 'and', 'doubted', 'and', 'dreamed', 'about', 'a', 'different', 'sort', 'of', 'life', '.', 'The', 'quiet', 'part', 'of', 'him', 'that', 'had', 'been', 'knocked', 'around', 'too', 'many', 'times', ',', 'so', 'it', 'got', 'locked', 'away', 'and', 'hidden', 'from', 'everyone', 'else', '.', '"', 'What', 'are', 'you', 'drawing', '?', '"', '"', 'Nothing', '.', '"', 'That', 'meant', 'he', 'did', "n't"]),
 Row(source='scribophile_texts1.txt', phrase=["'s", 'like', 'Buster', 'Keaton', 'without', 'the', 'gags', '.', 'Mrs.', 'Mac', 'says', 'he', 'has', 'no', 'bedside', 'manner', '

In [22]:
# label 1 = quality prose, 0 = crap prose
combined = spark.read.load('gs://project_joyce_scribo/phrases').withColumn('label', F.lit(0)) \
  .union(spark.read.load('gs://project_joyce/phrases').withColumn('label', F.lit(1))) \
  
collected = combined.sample(False, 0.1).rdd.map(lambda x : (x.phrase, x.label)).collect()



In [28]:
phrases = []
labels = []
random.shuffle(collected)
for phrase, label in collected:
  phrases.append(phrase)
  labels.append(label)

In [4]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(150000)
tokenizer.fit_on_texts(phrases)

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [25]:
len(phrases)

162223

In [39]:
tokenizer_bc = sc.broadcast(tokenizer)

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer

counter = CountVectorizer()
counter.fit(phrases[:100000])

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [44]:

lr = LogisticRegression(n_jobs=-1)
lr.fit(counter.transform(phrases[:100000]), labels[:100000])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [45]:
answers = lr.predict_proba(counter.transform(phrases[100000:]))[:, 1]

In [46]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
import numpy as np
log_loss(labels[100000:], answers)

0.4446631355550482

In [47]:
print(accuracy_score(labels[100000:], np.round(answers)))

0.7841473410153802


In [49]:
word_scores = np.array(sorted(zip(range(len(lr.coef_[0])), lr.coef_[0]), key=lambda x: abs(x[1]), reverse=True))
words = word_scores[:, 0]

In [56]:
counter_index = dict([(v, i) for i, v in counter.vocabulary_.items()])

In [58]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9879105640008058514
]


In [57]:
for word in words:
  print(counter_index[word])

brendon
laurence
temeraire
lenore
ron
harry
beckett
marek
yossarian
milkman
hermione
gately
suze
staryk
jem
sisky
willem
humans
amanda
snape
dumbledore
puss
jackie
cincinnatus
katharine
mavis
amory
gandalf
crumley
frodo
steinbeck
nate
kino
kasia
maruja
belacqua
jb
jonathan
jury
becky
jadine
lennie
scarcely
weasley
timothy
dude
mirnatius
ministry
corinthians
lorcan
lottie
lang
jody
douglas
amongst
ondine
iskierka
orin
lucas
voldemort
eli
lenz
marathe
mom
watt
hal
hagrid
heather
grimaced
yongxing
cade
andré
clarissa
greg
valerian
rocket
dorr
grin
rhiannon
malibu
olivia
aslan
abby
fredryk
cecilia
minnie
yanci
nigger
hogwarts
partially
marcus
kyle
brad
jude
colonel
ruthie
alright
narnia
macon
addiction
malachi
bea
mccain
vogue
cecy
kathy
kevin
arthos
mason
smirked
triumph
aragorn
tricia
papá
juana
chuckled
japhy
briony
reckoner
astonishment
estragon
snatch
nately
granby
hobbits
sethe
atwater
aureliano
hobbit
arcadio
lissy
luke
bloomwood
ganin
rattigan
reg
dalton
lizzie
narrowed
gryffindor


cuddles
radiantly
gondoliere
sari
tenement
tricked
scythe
thai
sammy
walkways
intriguing
simulated
galleon
aeron
persuasive
supplements
noose
decides
slamming
crinkled
publication
dreamed
elapsed
wielding
labs
hugging
viviane
attacker
kastos
shielded
diagonally
hustle
aura
twittering
dragons
sulky
questionable
chernobog
gazes
rowne
sounded
donna
temporal
parking
plucks
buzz
cooed
previously
processing
spooky
indulgent
ohio
adds
montreux
miser
stark
affliction
praises
imaginations
suspicion
describing
genny
furthermore
wooing
unnaturally
sends
vincent
broomsticks
realm
farmland
baying
picador
actions
heal
crucial
latches
veela
blinks
ferocity
humanitarian
goddess
phosphorus
absolute
matta
dostoevsky
harriett
lions
petal
lolling
fowl
hartford
talik
yams
tia
rectangle
exhilarating
shouts
artistic
swirling
general
mutton
fudge
stray
illusion
catriona
torrence
tosses
hermit
jac
uneven
offending
flexed
fifi
extremists
basics
meanest
lamb
ta
schtitt
dismal
impression
anne
hopped
stables
bullf

subjects
wyla
data
register
rocinante
unbelievable
fatty
blooded
daanya
contradicted
venus
polls
vellum
chad
immoenis
jon
parents
fellas
spiked
homely
tranquillity
hundreds
traitors
defenseless
sneaking
bending
schoolroom
bikes
flank
alexandria
landmark
fridays
davey
submissive
catalogs
contract
pallor
talios
celebratory
rapid
riddled
conductor
rolodex
mama
wielders
fontan
sclerosis
bedding
identified
brooke
equipage
withdraws
province
declan
yo
afresh
crevices
mutilation
hug
hewitt
buster
unison
correspondence
shameless
choice
jeering
aboriginal
twists
coupled
consortium
shrimp
glove
looming
influenced
maeve
korg
cataca
endowed
reminder
britton
forsaken
disloyalty
packages
yanko
commonly
portcullis
sera
disheartening
oughtn
fugue
underestimated
cocktail
maddie
grouping
pharmacist
maggie
bewildered
interpret
eponina
thornbury
mapped
axis
wayward
information
toga
robin
bashful
barely
neighbourhood
official
seized
rudely
irresistible
hurtful
foreseen
mantra
drips
underestimation
adversar

doze
backyard
belgian
worthington
esther
sayings
til
invent
coincidences
vision
imports
stealthy
duct
kites
mais
righted
joive
jiminy
delivering
ogres
sluiced
kaede
endlessness
mia
seam
wink
seagulls
fairies
yousoff
substances
gusts
acknowledged
romero
choked
florens
oven
brandenburg
shimmer
fluttering
attained
mathilda
attend
unstained
accumulating
clot
cowshed
flashing
scented
marhalt
rodrig
renovation
holies
noooo
excelled
bomb
confessor
exile
lien
yelping
orbital
achievement
hygienic
nightstand
elegance
greet
detestable
sending
hysterically
burton
coconuts
boudreau
peculiar
peat
ravi
unspoken
surrounded
dues
sunburned
festering
sparked
privet
olympus
cynical
anticipate
ow
reposed
hallmarks
pair
fathom
dismissal
dementors
reacted
bananas
revealing
nightgown
rae
napkins
wantin
nicest
borrow
livings
wheelbarrow
obligation
conceived
strutted
rats
cookies
cush
tiddlypush
threads
estelle
noticeably
quarts
spanish
cubes
hoyt
stairwell
void
footsore
rehearsed
bites
nap
puckered
maceo
neima

retriever
aggrieved
beware
poked
husband
editorial
davison
krishna
constructive
dig
gaping
spit
gemma
grimace
disappeared
tribulations
depicted
puppies
poetry
spinsters
poisoned
theater
bakelite
worshiped
rowan
ornaments
decanter
remade
sargon
moby
unnoticed
imbued
booze
parental
patsy
biddy
emptier
associations
donor
frighten
overexcited
horcrux
truce
siku
pinkie
reverberated
erich
tomlinson
idealism
grams
inclusive
compatriots
circuited
tobacco
blankness
squadrol
whether
essays
kehenrere
hectic
tendency
sanderson
charlestonwest
hitchhiking
sentiments
iain
totals
fumble
realities
florence
slumps
skepticism
heroically
fibroid
adding
consenting
seleen
requisite
headhunting
ring
linley
wheezes
pornographic
itchy
commissar
pointless
decatur
economist
jolts
staff
outrage
asphodel
robes
verdi
psychiatrists
loans
perchè
ajar
yon
assisting
selena
rattlesnake
mimicry
regained
jinny
bristly
guildhall
sedhik
soup
poodle
legacy
dora
addicted
mangoes
evangelists
restroom
elevate
petunias
walmart
p

...

dubarry
machiavelli
mamine
learn
apologies
mommeeeeeeeeeeee
epitaph
rockwells
melodeon
notepaper
310
topspin
unfairly
hp
brunswick
rolls
prescribed
purposelessly
plurabelle
razzle
baddeley
mannering
trollope
enmeshed
claimant
repartee
syne
tootsy
melinn
ahlan
more
drifter
plaited
buena
fountain
frizzed
floo
growth
hydroplaned
pituty
byconjuring
flemishartist
jumbocake
notvulgarize
sunriseward
committal
unspccomm
ahhhg
madre
desperate
fashionista
kaftans
orthopaedic
mettle
uncollected
13
bricked
thudded
golding
mcglome
nidorefne
olifar
rimen
zhioyan
corporations
clunking
rim
xp
bathes
infallibly
mcewan
dumpling
verbranden
blowhard
rooter
eyeballed
trampolines
knipperde
pirogue
poggin
gillend
kokhalzen
modder
modifier
emptiers
steals
singlet
georgette
victorian
snowman
sweetbreads
donated
priest
ode
gorra
aarp
unkillable
izbah
shchi
involve
hashing
doored
cathartic
vituperations
jocoseness
magstripe
dering
floun
celia
forearm
atwain
preying
undimmed
burro
solidities
949
gait
uneducated
c

wei
klutz
hermetism
logicians
mauthner
undirected
whitherward
aristos
skank
bai
speculate
811
rooo
wordsah
coeur
partied
voir
gotcha
drubbed
teetery
leakages
hors
stormsea
cecile
brainwashed
waisted
mizrez
zitten
unseasonably
ruts
pindar
lestocq
hanuman
residua
stokely
required
29th
rode
rainforests
midfielder
gladrags
wizardwear
cukor
cummerbund
fiberglass
filmmakers
murnau
méliès
syphoning
flusters
hearnshaw
maintains
putter
quiverfuls
gebeurd
gelopen
ontdekte
sporen
tijdlang
vermoord
vlammenwerper
brays
manchet
723
expatriation
naturalization
mellower
stouter
détente
adorning
roasts
dormancy
flamomer
gnarldrelif
conveyer
deranged
heartiness
tentando
archer
antonia
bursts
wilkens
hrt
conscientiousness
labime
coursed
geur
atenburn
indigent
gratify
slung
bein
epithet
asnug
bolshevist
nansenskiypasport
whoperceived
hamlin
unzips
judgment
katalina
bartowski
verder
slobber
cupola
oculist
beechen
biswa
lyangda
tents
dennison
kapitein
mullah
walpole
fingal
excelentísima
beuk
gieren
komen
la

weirdos
raillery
inheritors
décade
entretiens
transparently
suit
pinter
aprender
fealty
avondwind
kiezelstranden
kraakten
stoelen
zomen
sulay
brincadeira
clichês
conveniente
entretido
garoto
leito
naquilo
perdendo
poderia
verdadeiros
animator
pool
começando
crescer
fileira
longa
monstruosa
sul
neurosurgeons
alan
aficionado
afueros
trocchi
archus
gamp
nickerson
strabismic
obrero
luciana
feuds
broadcaster
etamine
injector
premonitory
moine
interrogations
playbook
puppet
moest
drafi
lorelei
incrível
ultimatums
unmarred
burúme
déagol
kamanda
lindor
dissect
overtipped
hedgefield
profane
publishes
marins
edgbaston
elphick
remonstrated
khemennu
fourier
furrier
ingen
uish
idara
especiais
reassessment
earing
fortyeight
tardies
rehabs
marasta
notoriously
dudeen
blackburn
fon
unrecorded
grond
transpo
incongruence
shahtoosh
verdomme
deckers
devorando
dissesse
fodam
insolência
obsceno
restaurante
nagnok
watchmaker
backtalk
fiddlestick
eternalized
vamoosed
bussed
stendhal
sitcoms
skittle
austria
fee

parroquia
warhound
zipster
refuser
ferthu
benzine
drinkt
ene
getijden
horden
langer
mensenmassa
naartoe
ongeduld
slapend
volgend
zwerven
ayyappa
apenas
quakes
uploading
obliviated
biplanes
sepulcher
acetylcysteine
inspissated
nebulizable
pronun
prophylaxis
urology
viscid
silêncio
chiding
twowhat
lataya
419
claxon
froid
manservants
waarop
casamento
inimiga
pois
quádruplo
satisfeita
secreting
psssst
dienst
hoef
roeren
verdrinkt
weigerde
laetificat
canopies
menagerie
athletics
besuited
glassheart
prowls
2956
2957
errantries
thorongil
siestas
soldiering
119
scamped
rationales
acome
alook
aust
lithen
dras
kalag
lecteur
nomale
habiting
pollitt
olhar
bantam
dimock
hows
roxbury
gladioli
rightness
suffusion
enfermé
elsamolay
oo
falar
turnin
bbc2
byline
gekend
korte
leven
uitstraalde
languor
coron
flourishes
ungovernable
altro
bicyclist
eeled
archived
valladolid
ket
ailes
anges
apollonius
avion
aéroplane
emplit
eucharistie
flottent
hostie
icare
joli
laisser
montent
prêtres
refermer
thyane
transp

acoquiner
deplorables
dise
excellentes
expressionistes
précézannienne
raisons
siècles
gambol
japes
méne
promis
rapports
doorhandles
515
bookman
lxxxvi
durmstrang
braiements
depuis
emouvants
exprimee
infiniment
nécessairement
prénapoléonique
sociaux
vertu
vulgarités
vécu
amarelos
baços
bolachas
bonecas
borracha
cobertos
confusos
expressões
farelentas
gordas
grossos
humana
leprosos
leveduras
maquiagem
pintados
quebradas
vomitada
barbe
longues
luisant
binnenviel
koude
opengeslagen
stikken
tast
voorwerp
enitharmon
peux
parselmouth
wrenchingly
quanto
screechy
textes
knotgrass
bouchée
congés
donnerai
moyen
musicien
payés
rimer
trompette
vite
épouvantable
diria
noção
quão
soubesse
xícara
bijvoorbeeld
denk
idee
kijkt
regel
tekst
toneel
wishart
chapado
controle
destroçados
explosões
lembro
maravilhosas
mentais
mescalina
milhões
peyotl
turpitude
criscross
malevolently
mandarins
approbation
gamekeeper
rté
haletant
parce
hamburgi
dmitri
dremus
scorning
dira
dérobade
formes
léobjet
prendront
selon
